In [34]:
import pandas as pd
import docx
import re

doc_path = 'LA.docx'

doc = docx.Document(doc_path)


In [ ]:
all_tables = []

for table in doc.tables:
    data = []
    for row in table.rows:
        data.append([cell.text.strip() for cell in row.cells])  # Extract text from each cell

    df = pd.DataFrame(data)  # Convert to DataFrame
    df.columns = df.iloc[0,:]
    df = df[1:]
    if df.shape[1] == 3:
        all_tables.append(df)

final_df = pd.concat(all_tables).reset_index(drop=True)

final_df['Descriptive Therapeutic Class'] = [re.sub(r' \(\d+\)', '', i) for i in final_df['Descriptive Therapeutic Class']]
final_df['Descriptive Therapeutic Class'] = ['' if re.search('[a-z]', i) else i for i in final_df['Descriptive Therapeutic Class']]
final_df['Descriptive Therapeutic Class'] = ['' if re.search('[*]', i) else i for i in final_df['Descriptive Therapeutic Class']]
final_df['Descriptive Therapeutic Class'] = [i.replace('\n', '') for i in final_df['Descriptive Therapeutic Class']]

final_df['Drugs on PDL'] = [i.replace('®', '') for i in final_df['Drugs on PDL']]
final_df['Drugs on PDL'] = [i.replace('NONE', '') for i in final_df['Drugs on PDL']]

final_df['Drugs on NPDL which Require Prior Authorization (PA)'] = [i.replace('®', '') for i in final_df['Drugs on NPDL which Require Prior Authorization (PA)']]
final_df['Drugs on NPDL which Require Prior Authorization (PA)'] = [i.replace('NONE', '') for i in final_df['Drugs on NPDL which Require Prior Authorization (PA)']]

final_df


,Descriptive Therapeutic Class,Drugs on PDL,Drugs on NPDL which Require Prior Authorization (PA)
0,"ACNE AGENTS, TOPICAL",Clindamycin/Benzoyl Peroxide Gel (Generic for ...,"Adapalene Cream, Gel Pump (Generic for Differin)"
1,,Clindamycin/Benzoyl Peroxide Gel (Generic for ...,Adapalene Gel (AG; Generic)
2,,Clindamycin Phosphate Gel (Generic),Adapalene/Benzoyl Peroxide (Generic for Epiduo)
3,,Clindamycin Phosphate Lotion (Generic),Adapalene/Benzoyl Peroxide Gel with Pump (AG; ...
4,,Clindamycin Phosphate Medicated Swab (Generic),Adapalene/Benzoyl Peroxide/Clindamycin Gel (Ca...
...,...,...,...
1731,,,
1732,UTERINE DISORDER TREATMENTS,Elagolix Tablet (Orilissa),
1733,,Elagolix/Estradiol/Norethindrone Capsule (Oria...,
1734,,Relugolix/Estradiol/Norethindrone Acetate (Myf...,


In [55]:
last_class = ''

for _, row in final_df.iterrows():
    if final_df.loc[_, 'Descriptive Therapeutic Class'] == '':
        final_df.loc[_, 'Descriptive Therapeutic Class'] = last_class
    else:
        last_class = final_df.loc[_, 'Descriptive Therapeutic Class']
    
final_df

,Descriptive Therapeutic Class,Drugs on PDL,Drugs on NPDL which Require Prior Authorization (PA)
0,"ACNE AGENTS, TOPICAL",Clindamycin/Benzoyl Peroxide Gel (Generic for ...,"Adapalene Cream, Gel Pump (Generic for Differin)"
1,"ACNE AGENTS, TOPICAL",Clindamycin/Benzoyl Peroxide Gel (Generic for ...,Adapalene Gel (AG; Generic)
2,"ACNE AGENTS, TOPICAL",Clindamycin Phosphate Gel (Generic),Adapalene/Benzoyl Peroxide (Generic for Epiduo)
3,"ACNE AGENTS, TOPICAL",Clindamycin Phosphate Lotion (Generic),Adapalene/Benzoyl Peroxide Gel with Pump (AG; ...
4,"ACNE AGENTS, TOPICAL",Clindamycin Phosphate Medicated Swab (Generic),Adapalene/Benzoyl Peroxide/Clindamycin Gel (Ca...
...,...,...,...
1731,UROLOGY INCONTINENCE,,
1732,UTERINE DISORDER TREATMENTS,Elagolix Tablet (Orilissa),
1733,UTERINE DISORDER TREATMENTS,Elagolix/Estradiol/Norethindrone Capsule (Oria...,
1734,UTERINE DISORDER TREATMENTS,Relugolix/Estradiol/Norethindrone Acetate (Myf...,


In [62]:
final_df.columns = ['therapeutic_class', 'Preferred', 'Non-Preferred']

df_melted = final_df.melt(
    id_vars=['therapeutic_class'],    
    value_vars=['Preferred', 'Non-Preferred'],    
    var_name='status',                        
    value_name='pdl_name'                       
)

df_melted = df_melted[df_melted['pdl_name'].str.strip() != ''].reset_index(drop=True)

df_melted

,therapeutic_class,status,pdl_name
0,"ACNE AGENTS, TOPICAL",Preferred,Clindamycin/Benzoyl Peroxide Gel (Generic for ...
1,"ACNE AGENTS, TOPICAL",Preferred,Clindamycin/Benzoyl Peroxide Gel (Generic for ...
2,"ACNE AGENTS, TOPICAL",Preferred,Clindamycin Phosphate Gel (Generic)
3,"ACNE AGENTS, TOPICAL",Preferred,Clindamycin Phosphate Lotion (Generic)
4,"ACNE AGENTS, TOPICAL",Preferred,Clindamycin Phosphate Medicated Swab (Generic)
...,...,...,...
2237,UROLOGY INCONTINENCE,Non-Preferred,"Solifenacin Tablet, Suspension (VESIcare; VESI..."
2238,UROLOGY INCONTINENCE,Non-Preferred,Tolterodine Tablet (Generic; Detrol)
2239,UROLOGY INCONTINENCE,Non-Preferred,Tolterodine ER Capsule (AG; Generic; Detrol LA)
2240,UROLOGY INCONTINENCE,Non-Preferred,"Trospium ER Capsule, Tablet (Generic)"
